In [2]:
# 기본 tool
import pandas as pd 
import numpy as np
import time
# 전처리
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
# PCA
from sklearn.decomposition import PCA 
from sklearn.preprocessing import scale
%matplotlib inline 

### 1. Data Import

In [3]:
train_rev = pd.read_pickle('train_rev')
tube_data = pd.read_pickle('tube_data')
bom_weight = pd.read_pickle('bom_weight')
test_rev = pd.read_pickle('test_rev')

### 2. Data Cleaning

#### (1) Dataframe 재정의

In [4]:
train_df = pd.merge(train_rev, tube_data, on ='tube_assembly_id', how = 'left')
train_df = pd.merge(train_df, bom_weight, on ='tube_assembly_id', how = 'left')

test_df = pd.merge(test_rev, tube_data, on ='tube_assembly_id', how = 'left')
test_df = pd.merge(test_df, bom_weight, on ='tube_assembly_id', how = 'left')

test에서 id를 별도 Series로 분리

In [5]:
test_id = test_df.iloc[:, 0].copy()
pd.to_pickle(test_id, 'test_id')

#### (2) Preprocessing

#### a. 결측값 제거

category 변수는 supplier, bracket_pricing, material_id, end_, forming_X, y 총 10개 존재함.  
- material_id 누락 -> 가장 많은 포션을 차지하는 SP-0029에 비하면 적은 양이나, 약 1%의 비중을 차지하는 부분임. 임의의 SP-9999 번호 부여

In [6]:
train_df['material_id'].fillna('SP-9999', inplace = True)
test_df['material_id'].fillna('SP-9999', inplace = True)

- 수치형 변수 diameter, wall, length, num_bends, bend_radius, num_boss, num_bracket, other 의 경우 평균값으로 채워줌

In [7]:
mean_col = ['diameter', 'wall', 'length', 'num_bends', 'bend_radius', 'num_boss', 'num_bracket', 'other']
for x in mean_col:
    train_df[x].fillna(train_df[x].mean(), inplace = True)
for x in mean_col:
    test_df[x].fillna(test_df[x].mean(), inplace = True)
# test_df에는 freq_supp 도 결측값이 있으므로 0로 채워줌
test_df['freq_supp'].fillna(0, inplace = True)

- Category 변수 end_a_1x, end_a_2x, end_x_1x, end_x_2x, forming_x, forming_y 의 경우 bfill로 임의의 값으로 대체

In [8]:
freq_col = ['end_a_1x', 'end_a_2x', 'end_x_1x', 'end_x_2x', 'forming_x', 'forming_y']
for x in freq_col:
    train_df[x].fillna(train_df[x].bfill(), inplace = True)
for x in freq_col:
    test_df[x].fillna(test_df[x].bfill(), inplace = True)

#### b. Label Encoding, Dummification

- catergory 변수 중 bracket_pricing, end_a_1x, end_a_2x, end_x_1x, end_x_2x, forming_x, y (7개)는 Y/N 이므로 Label Encoder를 활용해 0, 1로 변환

In [9]:
le = LabelEncoder()

# numpy array 형태로 변환
np_train_df = np.array(train_df)
np_test_df = np.array(test_df)

In [10]:
# bracket_pricing, end_a_1x, end_a_2x, end_x_1x, end_x_2x, forming_x, y 변환
for i in range(np_train_df.shape[1]):
    if i in [3, 15, 16, 17, 18, 22, 23]:
        np_train_df[:,i] = le.fit_transform(np_train_df[:,i])

for i in range(np_test_df.shape[1]):
    if i in [4, 15, 16, 17, 18, 22, 23]:
        np_test_df[:, i] = le.fit_transform(np_test_df[:,i])

le_train_df = pd.DataFrame(np_train_df)
le_train_df.columns = train_df.columns

le_test_df = pd.DataFrame(np_test_df)
le_test_df.columns = test_df.columns

- category 변수 중 material_id의 value가 여러가지 이므로 dummy화하고, supplier, tube_assembly_id 정보는 제거

In [11]:
dum_mat_col = ['material_id']
dum_mat = pd.get_dummies(le_train_df[dum_mat_col])
dum_mat_t = pd.get_dummies(le_test_df[dum_mat_col])

dum_mat_t['material_id_SP-0030'] = pd.Series(0)
dum_mat_t['material_id_SP-0031'] = pd.Series(0)
dum_mat_t= dum_mat_t.fillna(0)

processed_train_df = pd.concat([le_train_df, dum_mat], axis=1, join_axes=[le_train_df.index])
processed_train_df = processed_train_df.drop(['supplier', 'tube_assembly_id', 'material_id'], axis = 1)

processed_test_df = pd.concat([le_test_df, dum_mat_t], axis=1, join_axes=[le_test_df.index])
processed_test_df = processed_test_df.drop(['supplier', 'tube_assembly_id', 'material_id'], axis = 1)

#### c. outlier 제거

EDA에서 확인한 length 0, bend_radius 9999 은 이상치로 간주하여 제거 (단, train set에서만)

In [12]:
processed_train_df = processed_train_df.loc[processed_train_df['length'] > 0]
processed_train_df = processed_train_df.loc[processed_train_df['bend_radius'] < 9999]

#### (3) 학습을 위한 최종 data set 저장

train set에서 종속 변수 cost 별도 DF으로 분리하고, test set에서 모델 학습에 불필요한 id 변수 제거

In [13]:
df_y = processed_train_df.iloc[:, 2].copy()
processed_train_df = processed_train_df.drop(['cost'], axis=1)
processed_test_df = processed_test_df.drop(['id'], axis=1)

model 학습을 위해 numeric 변수로 변환

In [14]:
fin_train_df = processed_train_df.apply(pd.to_numeric)
df_y = df_y.apply(pd.to_numeric)
fin_test = processed_test_df.apply(pd.to_numeric)

In [15]:
# for RMSLE 로 학습하기 위해 log (x + 1) 형태로 변환
log_df_y = np.log1p(df_y) 

XGBoost model 학습을 위해 train, test의 column order를 맞춰줌

In [25]:
fin_train_df1= fin_train_df.reindex_axis(sorted(fin_train_df.columns), axis=1)
fin_test1= fin_test.reindex_axis(sorted(fin_train_df.columns), axis=1)

정리된 DataFrame을 csv 형태로 저장

In [28]:
fin_train_df1.to_csv('my_train1.csv', header = True, index = False)
log_df_y.to_csv('my_log_df_y1.csv', header = True, index = False)
fin_test1.to_csv('my_test1.csv', header = True, index = False)